In [ ]:
# Make sure to have the add-on "ZMQ remote API"
# running in CoppeliaSim
# All CoppeliaSim commands will run in blocking mode (block
# until a reply from CoppeliaSim is received). For a non-
# blocking example, see simpleTest-nonBlocking.py
# can check all regular API https://www.coppeliarobotics.com/helpFiles/en/apiFuncti
import time
import math 
from zmqRemoteApi import RemoteAPIClient
import numpy as np
print('Program started')
client = RemoteAPIClient()
sim = client.getObject('sim')
simIK= client.getObject('simIK')
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default. So let's make
# sure that the idle loop runs at full speed for this program:
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)
sim.setInt32Param(sim.intparam_idle_fps, 0)

In [ ]:
## write Hello on program status bar
sim.addLog(sim.verbosity_scriptinfos, "Hello")
# Run a simulation in asynchronous mode:
client.setStepping(False)
sim.startSimulation()

In [ ]:
## Your code starts here==>>>
import time
from zmqRemoteApi import RemoteAPIClient
import numpy as np


##Name: Omar Khaled Mostafa
##ID: 171033
## Your conde ends here <<<<==
##Initialize basic values:
jointHandles={}
index=0;
for i in 1,7,1:
    jointHandles[i]=sim.getObject('./joint',[index==i-1])
    break

##nitializing Time for KUKA14A
startTime=sim.getSimulationTime()
endTime= startTime+30
deltaT_insert= 3
##Initializing Handles for KUKA14A

simBaseA=sim.getObject('/KUKA14A')
##simBaseB=sim.getObject('.KUKA14B')
simGrippperA=sim.getObject('./RG2A')
##simGrippperB=sim.getObject('./RG2B')
simTipA=sim.getObject('./TipA')
##simTipB=sim.getObject('./TipB')    
simTargetA=sim.getObject('/TargetA')
simTargetB=sim.getObject('/TargetB')
simConveyorA=sim.getObject('/conveyorA')
simConveyorB=sim.getObject('/conveyorB')
simConveyorC=sim.getObject('/conveyorC')
simSensorA=sim.getObject('/proxSensorA')
simSensorB=sim.getObject('/proxSensorB')
simSensorC=sim.getObject('/proxSensorC')
simCuboid=sim.getObject('/Cuboid')
##Initializing Orientations for KUKA14A
targetStartPositionA={-0.915,-0.425,1.155}
targetEndPositionA={-0.915,0.4802,1.1282}
targetNextPositionA={-0.425,0.575,1.155}
##targetPosition=sim.getObjectPosition(simCuboid,-1)
##targetPosition[3]=targetPosition[3]+0.1
##desiredOrientation={0*math.pi/180,0*math.pi/180,135*math.pi/180}
##cuboidQuaternion=sim.getObjectQuaternion(simCuboid,-1)
##Inverse Kinamatics initialization for KUKA14A
ikEnv=simIK.createEnvironment()##create an IK environment
    ##Undamped set-up
ikGroup_undamped=simIK.createIkGroup(ikEnv) ##create an IK group
simIK.setIkGroupCalculation(ikEnv,ikGroup_undamped,simIK.method_pseudo_inverse,0,6) ##set its resolution method to undamped
simIK.addIkElementFromScene(ikEnv,ikGroup_undamped,simBaseA,simTipA,simTargetA,simIK.constraint_pose) ##create an IK element based on the scene content
    ##Damped set-up
ikGroup_damped=simIK.createIkGroup(ikEnv) ##create another IK group
simIK.setIkGroupCalculation(ikEnv,ikGroup_damped,simIK.method_damped_least_squares,1,99) ##set its resolution method to damped
simIK.addIkElementFromScene(ikEnv,ikGroup_damped,simBaseA,simTipA,simTargetA,simIK.constraint_pose) ##create an IK element based on the scene content
##Constants for KUKA14A
vel=1  
accel=0.5
jerk=0.5
maxVel={vel,vel,vel,50*math.pi/180}
maxAccel={accel,accel,accel,20*math.pi/180}
maxJerk={jerk,jerk,jerk,20*math.pi/180}
while(sim.getSimulationTime()<endTime):
    ##pos=math.random(1,3)
    color = {1,0,0}
    ##color[pos] = 1
    proxBReading=sim.readProximitySensor(simSensorB)
    if (sim.getSimulationTime() > (startTime + deltaT_insert) and proxBReading != 1):
        detectedObjectHandle=sim.createPureShape(0,2+8,{0.06,0.06,0.06},0.1,nil)
        sim.setShapeColor(detectedObjectHandle,nil,sim.colorcomponent_ambient_diffuse,color)
        sim.setObjectPosition(detectedObjectHandle,-1,targetStartPositionA)
        deltaT_insert=deltaT_insert+3
        sim.setObjectSpecialProperty(detectedObjectHandle,sim.objectspecialproperty_measurable+sim.objectspecialproperty_detectable+sim.objectspecialproperty_collidable)

    if proxBReading == 1:
        sim.writeCustomTableData(simConveyorB,'__ctrl__',[vel==0])
        sim.wait(5)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        cuboidQuaternion=sim.getObjectQuaternion(simCuboid,-1)
        ##targetPosition=sim.getObjectPosition(simCuboid,-1)
        targetPosition= targetEndPositionA
        targetPosition[3]=targetPosition[3]+0.1
        sim.rmlMoveToPosition(simTargetA,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.wait(3)
        sim.setInt32Signal('RG2A_open',1)
        targetPosition[3]=targetPosition[3]-0.1
        sim.rmlMoveToPosition(simTargetA,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.setInt32Signal('RG2A_open',0)
        sim.wait(3)
        targetPosition[3]=targetPosition[3]+0.2
        sim.rmlMoveToPosition(simTargetA,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.writeCustomTableData(simConveyorB,'__ctrl__',[vel==0.1])
        targetPosition= targetNextPositionA
        sim.rmlMoveToPosition(simTargetA,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.wait(3)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.setInt32Signal('RG2A_open',0)
        sim.setInt32Signal('RG2A_open',1)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        ##else
            ##sim.writeCustomTableData(simConveyorB,'__ctrl__',{vel=0.1})
        ##end
            ##This proximity sensor serves as an emergency stop for the entire system 
        ##if proxCReading == 1 then
        ##    print('Wrong color detected')
        ##    print('System Shutdown')
        ##    sim.writeCustomTableData(simConveyorA,'__ctrl__',{vel=0})
        ##    sim.writeCustomTableData(simConveyorB,'__ctrl__',{vel=0})
        ##    sim.writeCustomTableData(simConveyorC,'__ctrl__',{vel=0})
        ##end
    break

In [ ]:
jointHandles={}
index=0;
for i in 1,7,1:
    jointHandles[i]=sim.getObject('./joint',[index==i-1])
    break
##Initializing Time for KUKA14B
startTime=sim.getSimulationTime()
endTime= startTime+30
##Initializing Time for KUKA14B
startTime=sim.getSimulationTime()
endTime= startTime+30
##Initializing Handles for KUKA14B
##simBaseA=sim.getObject('.')
simBaseB=sim.getObject('./KUKA14B')
##simGrippperA=sim.getObject('./RG2A')
simGrippperB=sim.getObject('./RG2B')
##simTipA=sim.getObject('./TipA')
simTipB=sim.getObject('./TipB')    
simTargetA=sim.getObject('/TargetA')
simTargetB=sim.getObject('/TargetB')
simConveyorA=sim.getObject('/conveyorA')
simConveyorB=sim.getObject('/conveyorB')
simConveyorC=sim.getObject('/conveyorC')
simSensorA=sim.getObject('/proxSensorA')
simSensorB=sim.getObject('/proxSensorB')
simSensorC=sim.getObject('/proxSensorC')
simCuboid=sim.getObject('/Cuboid')
##Initializing Orientations for KUKA14B
targetNextPositionA={-0.425,0.575,1.155}
targetStartPositionB= targetNextPositionA
##targetStartPositionB={-0.425,0.575,1.155}
targetEndPositionB={0.4952,0.665,1.128}
targetNextPosition={0.875,0.465,1.155}
##targetPosition=sim.getObjectPosition(simCuboid,-1)
##targetPosition[3]=targetPosition[3]+0.1
##desiredOrientation={0*math.pi/180,0*math.pi/180,135*math.pi/180}
##cuboidQuaternion=sim.getObjectQuaternion(simCuboid,-1)
##Inverse Kinamatics initialization for KUKA14B
ikEnv=simIK.createEnvironment()##create an IK environment
    ##Undamped set-up
ikGroup_undamped=simIK.createIkGroup(ikEnv)##create an IK group
simIK.setIkGroupCalculation(ikEnv,ikGroup_undamped,simIK.method_pseudo_inverse,0,6)##set its resolution method to undamped
simIK.addIkElementFromScene(ikEnv,ikGroup_undamped,simBaseB,simTipB,simTargetB,simIK.constraint_pose)##create an IK element based on the scene content
    ##Damped set-up
ikGroup_damped=simIK.createIkGroup(ikEnv)##create another IK group
simIK.setIkGroupCalculation(ikEnv,ikGroup_damped,simIK.method_damped_least_squares,1,99)##set its resolution method to damped
simIK.addIkElementFromScene(ikEnv,ikGroup_damped,simBaseB,simTipB,simTargetB,simIK.constraint_pose)##create an IK element based on the scene content
##Constants for KUKA14B
vel=1  
accel=0.5
jerk=0.5
maxVel={vel,vel,vel,50*math.pi/180}
maxAccel={accel,accel,accel,20*math.pi/180}
maxJerk={jerk,jerk,jerk,20*math.pi/180}
##Main Loop for KUKA14B    
while(sim.getSimulationTime()<endTime):
    proxAReading=sim.readProximitySensor(simSensorA)
    
    if proxAReading != 1: 
        sim.writeCustomTableData(simConveyorA,'__ctrl__',[vel==0.1])
       
    if proxAReading == 1:
        sim.writeCustomTableData(simConveyorA,'__ctrl__',[vel==0])
        sim.wait(5)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        cuboidQuaternion=sim.getObjectQuaternion(simCuboid,-1)
        targetPosition=targetEndPositionB
        targetPosition[3]=targetPosition[3]+0.1
        sim.rmlMoveToPosition(simTargetB,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.wait(3)
        sim.setInt32Signal('RG2B_open',1)
        targetPosition[3]=targetPosition[3]-0.1
        sim.rmlMoveToPosition(simTargetB,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.setInt32Signal('RG2B_open',0)
        sim.wait(3)
        targetPosition[3]=targetPosition[3]+0.2
        sim.rmlMoveToPosition(simTargetB,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        targetPosition= targetNextPosition
        sim.rmlMoveToPosition(simTargetB,-1,-1,nil,nil,maxVel,maxAccel,maxJerk,targetPosition,cuboidQuaternion,nil)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.wait(3)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.setInt32Signal('RG2B_open',0)
        sim.setInt32Signal('RG2B_open',1)
        simIK.applyIkEnvironmentToScene(ikEnv,ikGroup_damped)
        sim.writeCustomTableData(simConveyorB,'__ctrl__',[vel==0.1])
##This proximity sensor serves as an emergency stop for the entire system 
##if proxCReading == 1 then
## print('Wrong color detected')
##print('System Shutdown')
##sim.writeCustomTableData(simConveyorA,'__ctrl__',{vel=0})
##sim.writeCustomTableData(simConveyorB,'__ctrl__',{vel=0})
##sim.writeCustomTableData(simConveyorC,'__ctrl__',{vel=0})


In [ ]:
##sim.stopSimulation()
# Restore the original idle loop frequency:
##sim.setInt32Param(sim.intparam_idle_fps, defaultIdleFps)
##print('Program ended')